In [0]:
# LEARNING SYSTEM
# PREPROCESSING CODE

In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
import sklearn
import torch
import os
import re

In [0]:
# global variables
path_to_txt_data = ""
saved_path = "" # path to saved pytorch objects

k = 300 # out1 layer of first neural network
c = 300 # out2 layer of first neural network

h = 50  # out1 layer of second neural network
u = 50  # out2 layer of second neural network

In [0]:
def preprocessing(path_to_txt):
    df = pd.DataFrame(columns=['text', 'label']) # define dataframe
    for files in os.walk('{}\ham'.format(path)): # read all files with path .\ham
        for i in files[2]:
            try:
                f = open('{}\ham'.format(path)+'\\'+i, newline=None)
                sp = f.read()
                sp = re.sub( r'[^a-z 0-9!.,$%&?"]', '', sp) # symbols from Table 2
                df = df.append({'text':sp, 'label':0}, ignore_index=True)
                f.close()
            except: pass
    for files in os.walk('{}\spam'.format(path)): # read all files with path .\spam
        for i in files[2]:
            try:
                f = open('{}\spam'.format(path)+'\\'+i, newline=None)
                sp = f.read()
                sp = re.sub( r'[^a-z 0-9!.,$%&?"]', '', sp) # symbols from Table 2
                df = df.append({'text':sp, 'label':1}, ignore_index=True)
                f.close()
            except: pass
    return df

In [0]:
# feature selection function

In [0]:
def latter_encoder(x):
    if x == 'a':
        return 65
    elif x == 'b':
        return 66
    elif x == 'c':
        return 67
    elif x == 'd':
        return 68
    elif x == 'e':
        return 69
    elif x == 'f':
        return 70
    elif x == 'g':
        return 71
    elif x == 'h':
        return 72
    elif x == 'i':
        return 73
    elif x == 'j':
        return 74
    elif x == 'k':
        return 75
    elif x == 'l':
        return 76
    elif x == 'm':
        return 77
    elif x == 'n':
        return 78
    elif x == 'o':
        return 79
    elif x == 'p':
        return 80
    elif x == 'q':
        return 81
    elif x == 'r':
        return 82
    elif x == 's':
        return 83
    elif x == 't':
        return 84
    elif x == 'u':
        return 85
    elif x == 'v':
        return 86
    elif x == 'w':
        return 87
    elif x == 'x':
        return 88
    elif x == 'y':
        return 89
    elif x == 'z':
        return 90
    elif x == ' ':
          return 32
    elif x == '!':
          return 33
    elif x == '"':
          return 34
    elif x == '#':
          return 35
    elif x == '$':
          return 36
    elif x == '%':
          return 37
    elif x == '&':
          return 38
    elif x == "'":
          return 39
    elif x == '(':
          return 40
    elif x == ')':
          return 41
    elif x == '*':
          return 42
    elif x == '+':
          return 43
    elif x == ',':
          return 44
    elif x == '-':
          return 45
    elif x == '.':
          return 46
    elif x == '/':
          return 47
    elif x == '0':
          return 48
    elif x == '1':
          return 49
    elif x == '2':
          return 50
    elif x == '3':
          return 51
    elif x == '4':
          return 52
    elif x == '5':
          return 53
    elif x == '6':
          return 54
    elif x == '7':
          return 55
    elif x == '8':
          return 56
    elif x == '9':
          return 57
    elif x == ':':
          return 58
    elif x == ';':
          return 59
    elif x == '<':
          return 60
    elif x == '=':
          return 61
    elif x == '>':
          return 62
    elif x == '?':
          return 63
    elif x == '@':
          return 64
    else: return 91

def feature_selection(doc):
    res = []
    for i in range(len(doc)):
        res.append((latter_encoder(doc[i])-32)/(91-32))
    return res

In [0]:
# Neural network for text presentation

In [0]:
class Net(nn.Module):
    def __init__(self, Input, k, c):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(Input, k)
        self.fc2 = nn.Linear(k, c)
        self.fc3 = nn.Linear(c, Input)
        self.fc1.bias = None
        self.fc2.bias = None
        self.fc3.bias = None
        torch.nn.init.constant_(self.fc1.weight, 0.01)
        torch.nn.init.constant_(self.fc2.weight, 0.01)
        weights = torch.empty(2000, 2000)
        torch.manual_seed(101)
        torch.nn.init.xavier_uniform_(weights, gain=1)
        self.fc3.weight.data = weights[:Input, :c]
    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x
    def answer(self, x):
        x = torch.sigmoid(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

In [0]:
# Learning of fisrt neural network with SGD algorithm

In [0]:
def learningFirstNn(df, c):
    res = []
    for ind in len(df):
        vec = torch.Tensor(feature_selection(df.iloc[ ind, 0]))
        net = Net(len(vec), c)
        net = net
        vec = vec
        loss_fn = torch.nn.MSELoss()
        learning_rate = 3
        optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)
        counter = 0
        loss_pred = 0
        for t in range(2000):
            y_pred = net(vec)
            loss = loss_fn(y_pred, vec)
            if loss_pred == loss.data:
                counter+=1
                learning_rate = learning_rate/2
                optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)
            if counter >= 2:
                counter = 0 
                learning_rate = 5
                optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)
            if loss < 10**(-3):
                print('{} {}'.format(ind, loss.data))
                break
            if t == 1999:
                print('{} {}'.format(ind, loss.data))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_pred = loss.data
        try:
            res.append(torch.cat((net.answer(vec), torch.Tensor([df.iloc[ind, 1]]))))
        except: 
            res.append(net.answer(vec))
        return res

In [0]:
# save results to file

In [0]:
def saveToFile(saved_path, torchObj):
    torch.save(torchObj, saved_path)

In [0]:
# fit scaler

In [0]:
def scaler_learn(res):
    scaler = MinMaxScaler()
    scaler.fit(res)
    # use scaler.transform(vector) to apply scalling
    return scaler 

In [0]:
#######################
#classification models#
#######################

In [0]:
# neural network classifier

In [0]:
class Net(nn.Module):
    def __init__(self, c, h, u):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(c, h)
        self.dout1 = nn.Dropout(0.1)
        self.fc2 = nn.Linear(h, u)
        self.dout2 = nn.Dropout(0.1)
        self.fc3 = nn.Linear(u, 1)
        torch.nn.init.xavier_uniform_(self.fc1.weight.data, gain=1)
        torch.nn.init.xavier_uniform_(self.fc2.weight.data, gain=1)
        torch.nn.init.xavier_uniform_(self.fc3.weight.data, gain=1)
        self.fc1.bias = None
        self.fc2.bias = None
        self.fc3.bias = None
    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        x = self.dout1(x)
        x = torch.sigmoid(self.fc2(x))
        x = self.dout2(x)
        x = torch.sigmoid(self.fc3(x))
        return x

In [0]:
def learningSecondNn(res, c, h, u):
    net = Net(c, h, u)
    res = res[torch.randperm(res.size()[0])]
    X_train, X_test, y_train, y_test = res[:6000, :-1], res[6000:, :-1], res[:6000, -1], res[6000:, -1]
    loss_fn = torch.nn.MSELoss()
    learning_rate = 0.01
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
    num_epochs = 2000
    X = X_train
    y = y_train
    for t in range(num_epochs):
          X = Variable(X)
          y = Variable(y)
          y_pred = net(X)
          loss = loss_fn(y_pred, y.view(-1,1))
          optimizer.zero_grad()
          loss.backward(retain_graph=True)
          optimizer.step()
          print('epoch {}'.format(t), ' ',loss.data)
          if loss.data < 0.1:
            break
    return net

In [0]:
# naive Bayes classifier

In [0]:
def nbc(res):
    X_train, X_test, y_train, y_test = train_test_split(res[:,:-1], res[:,-1], test_size=0.15)
    clf = GaussianNB()
    clf.fit(X_train, y_train)
    return clf

In [0]:
## other models

In [0]:
# logistic regression

In [0]:
solver = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
max_iter = [500, 1000]
for i in solver:
    X_train, X_test, y_train, y_test = train_test_split(res[:,:-1], res[:,-1], test_size=0.15)
    clf = LogisticRegression(max_iter=1000, solver=i).fit(X_train, y_train)
    m = clf.score(X_test, y_test)
    print(m, ' ',i)

In [0]:
# random forest 

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
mypipeline = Pipeline([
('scaler', minmaxScaler()),
('forest', RandomForestClassifier())
])
params = {
'forest__n_estimators': range(90, 120),
'forest__min_samples_split': range(4, 7),
'forest__min_samples_leaf': [1, 2],
'forest__max_features': [50, 60],
'forest__max_depth': range(40, 80) }

In [0]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=101)
hyper_search = RandomizedSearchCV(mypipeline, param_grid_base, n_iter=5, scoring='accuracy',
cv=cv, n_jobs=-1, refit=True, random_state=101,
verbose=2)
hyper_search.fit(X_train, y_train)

In [0]:
## metrics ## 

In [0]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
def printMetrics(y_pred, y):
    print(confusion_matrix(y,y_pred))
    print(classification_report(y,y_pred))
    print(accuracy_score(y, y_pred))

In [0]:
#######################
#######################
#classification system#

In [0]:
def myClassifier(txt_obj, scaler, net):
    df = pd.DataFrame(columns=['text', 'label'])
    df.append({'text':txt_obj, 'label':'Unknown'}, ignore_index=True)
    vector = learningFirstNn(df, 300)
    vector = scaler.transform(vector)
    print('The answer of the classifier is ', net(vector))